In [108]:
# python native
import os
import json
import random
import datetime
from functools import partial

# external library
import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.model_selection import GroupKFold
import albumentations as A

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models

# visualization
import matplotlib.pyplot as plt

import os
import cv2
import numpy as np
from tensorflow import keras



#경로설정
SAVED_DIR = "./workspace/results_baseline/"
IMAGE_ROOT = "/opt/ml/input/data/test/DCM"
CLASSES = [
    'finger-1', 'finger-2', 'finger-3', 'finger-4', 'finger-5',
    'finger-6', 'finger-7', 'finger-8', 'finger-9', 'finger-10',
    'finger-11', 'finger-12', 'finger-13', 'finger-14', 'finger-15',
    'finger-16', 'finger-17', 'finger-18', 'finger-19', 'Trapezium',
    'Trapezoid', 'Capitate', 'Hamate', 'Scaphoid', 'Lunate',
    'Triquetrum', 'Pisiform', 'Radius', 'Ulna',
]
CLASS2IND = {v: i for i, v in enumerate(CLASSES)}
IND2CLASS = {v: k for k, v in CLASS2IND.items()}

In [109]:
#학습한 모델들 불러오기 (왼손,오른손)
modelR = torch.load(os.path.join(SAVED_DIR, "fcn_resnet50_best_model_Right.pt"))
modelL= torch.load(os.path.join(SAVED_DIR, "fcn_resnet50_best_model_Left.pt"))
#LR분류모델
modelLR = keras.models.load_model('./workspace/results_baseline/my_model.h5')

In [125]:
pngs = {
    os.path.relpath(os.path.join(root, fname), start=IMAGE_ROOT)
    for root, _dirs, files in os.walk(IMAGE_ROOT)
    for fname in files
    if os.path.splitext(fname)[1].lower() == ".png"
}


In [111]:
#classification model
model = keras.models.load_model('my_model.h5')

test_dir = '/opt/ml/input/data/test/DCM'

predictions = []
pngsL=set()
pngsR=set()
for i in range(1, 600):
    folder_name = f"ID{i:03d}"
    folder_path = os.path.join(test_dir, folder_name)
    
    if not os.path.exists(folder_path):
        continue  # 폴더가 존재하지 않으면 다음 폴더로 건너뜁니다
    
    image_paths = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.png'):
            image_paths.append(os.path.join(folder_path, file_name))
            # image_paths.append(folder_name+'/'+file_name)
    
    for image_path in image_paths:
        image = cv2.imread(image_path)
        image = cv2.resize(image, (224, 224))  # 이미지 크기 조정
        image = image.astype("float32") / 255.0  # 이미지 스케일링
        image = np.expand_dims(image, axis=0)  # 배치 차원 추가
        
        prediction = model.predict(image)
        prediction_label = "Left" if prediction[0][0] > 0.5 else "Right"
        if prediction_label=="Left":
            pngsL.add(image_path[28:])
        if prediction_label=="Right":
            pngsR.add(image_path[28:])
        predictions.append((image_path, prediction_label,prediction[0][0]))
        


1/1 [==============================] - 0s 29ms/step


In [112]:
# print(pngsL)
# print('--------')
# print(pngsR)

In [113]:
# mask map으로 나오는 인퍼런스 결과를 RLE로 인코딩 합니다.

def encode_mask_to_rle(mask):
    '''
    mask: numpy array binary mask 
    1 - mask 
    0 - background
    Returns encoded run length 
    '''
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [114]:
# RLE로 인코딩된 결과를 mask map으로 복원합니다.

def decode_rle_to_mask(rle, height, width):
    s = rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(height * width, dtype=np.uint8)
    
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    
    return img.reshape(height, width)

In [115]:
class XRayInferenceDataset(Dataset):
    def __init__(self, transforms=None):
        _filenames = pngsR
        _filenames = np.array(sorted(_filenames))
        
        self.filenames = _filenames
        self.transforms = transforms
    
    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, item):
        image_name = self.filenames[item]
        image_path = os.path.join(IMAGE_ROOT, image_name)
        
        image = cv2.imread(image_path)
        image = image / 255.
        
        if self.transforms is not None:
            inputs = {"image": image}
            result = self.transforms(**inputs)
            image = result["image"]

        # to tenser will be done later
        image = image.transpose(2, 0, 1)    # make channel first
        
        image = torch.from_numpy(image).float()
            
        return image, image_name

In [116]:
def test(model, data_loader,rles=[],filename_and_class=[], thr=0.5):
    model = model.cuda()
    model.eval()


    with torch.no_grad():
        n_class = len(CLASSES)

        for step, (images, image_names) in tqdm(enumerate(data_loader), total=len(data_loader)):
            images = images.cuda()    
            outputs = model(images)['out']
            
            # restore original size
            outputs = F.interpolate(outputs, size=(2048, 2048), mode="bilinear")
            outputs = torch.sigmoid(outputs)
            outputs = (outputs > thr).detach().cpu().numpy()
            
            for output, image_name in zip(outputs, image_names):
                for c, segm in enumerate(output):
                    rle = encode_mask_to_rle(segm)
                    rles.append(rle)
                    filename_and_class.append(f"{IND2CLASS[c]}_{image_name}")
                    
    return rles, filename_and_class


In [117]:
tf = A.Resize(512, 512)
test_dataset = XRayInferenceDataset(transforms=tf)
test_loader = DataLoader(
    dataset=test_dataset, 
    batch_size=2,
    shuffle=False,
    num_workers=2,
    drop_last=False
)
#print(list(test_dataset)[0])

In [118]:
rles, filename_and_class = test(modelR, test_loader)

class XRayInferenceDataset(Dataset):
    def __init__(self, transforms=None):
        _filenames = pngsL
        _filenames = np.array(sorted(_filenames))
        
        self.filenames = _filenames
        self.transforms = transforms
    
    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, item):
        image_name = self.filenames[item]
        image_path = os.path.join(IMAGE_ROOT, image_name)
        
        image = cv2.imread(image_path)
        image = image / 255.
        
        if self.transforms is not None:
            inputs = {"image": image}
            result = self.transforms(**inputs)
            image = result["image"]

        # to tenser will be done later
        image = image.transpose(2, 0, 1)    # make channel first
        
        image = torch.from_numpy(image).float()
            
        return image, image_name
test_dataset = XRayInferenceDataset(transforms=tf)
test_loader = DataLoader(
    dataset=test_dataset, 
    batch_size=2,
    shuffle=False,
    num_workers=2,
    drop_last=False
)
rles, filename_and_class = test(modelL, test_loader,rles,filename_and_class)
#print(filename_and_class)

In [119]:
print(filename_and_class[0])
print(filename_and_class[1])
print(rles[0])

finger-1_ID040/image1661319116107.png
finger-2_ID040/image1661319116107.png
1814994 2 1817041 6 1819087 11 1821135 15 1823182 20 1825230 25 1827278 32 1829325 35 1831373 36 1833421 38 1835469 39 1837516 40 1839564 41 1841612 42 1843660 43 1845708 43 1847756 44 1849803 46 1851851 47 1853899 48 1855947 49 1857995 49 1860043 50 1862091 51 1864139 52 1866187 53 1868235 53 1870283 54 1872331 54 1874380 53 1876428 54 1878477 53 1880525 53 1882574 53 1884623 52 1886672 52 1888721 51 1890770 50 1892819 50 1894869 48 1896919 47 1898968 47 1901018 45 1903067 45 1905116 45 1907165 45 1909214 45 1911263 45 1913312 45 1915361 45 1917409 46 1919458 47 1921507 47 1923555 48 1925604 47 1927652 48 1929701 48 1931750 47 1933798 48 1935847 48 1937895 48 1939944 48 1941993 48 1944041 48 1946090 48 1948139 48 1950187 49 1952236 49 1954284 50 1956333 50 1958382 49 1960430 50 1962479 50 1964528 50 1966576 51 1968625 51 1970674 52 1972723 52 1974771 53 1976820 53 1978869 53 1980918 54 1982966 55 1985015 55 19

In [120]:
classes, filename = zip(*[x.split("_") for x in filename_and_class])
image_name = [os.path.basename(f) for f in filename]
df = pd.DataFrame({
    "image_name": image_name,
    "class": classes,
    "rle": rles,
})
df.to_csv("output.csv", index=False)